In [1]:
import os
import h5py
from sklearn.cross_validation import StratifiedKFold
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, \
Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

from convnetskeras.customlayers import convolution2Dgroup, crosschannelnormalization, \
splittensor, Softmax4D

from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop
from sklearn import svm
from keras.utils import np_utils
import numpy as np

import sys, glob, argparse
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

try:
    import cPickle as pickle
except:
    import pickle

batch_size = 32
nb_classes = 10
nb_epoch = 200
data_augmentation = False

# input image dimensions
img_rows, img_cols = 227,227
img_channels = 3

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.jet):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(10)
    plt.xticks(tick_marks, ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
 'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi'], rotation=45)
    plt.yticks(tick_marks, ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
 'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi'])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def visualize_loss(hist):
    train_loss=hist.history['loss']
    val_loss=hist.history['val_loss']
    train_acc=hist.history['acc']
    val_acc=hist.history['val_acc']
    xc=range(nb_epoch)

    plt.figure(1,figsize=(7,5))
    plt.plot(xc,train_loss)
    plt.plot(xc,val_loss)
    plt.xlabel('num of Epochs')
    plt.ylabel('loss')
    plt.title('train_loss vs val_loss')
    plt.grid(True)
    plt.legend(['train','val'])
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])

    plt.figure(2,figsize=(7,5))
    plt.plot(xc,train_acc)
    plt.plot(xc,val_acc)
    plt.xlabel('num of Epochs')
    plt.ylabel('accuracy')
    plt.title('train_acc vs val_acc')
    plt.grid(True)
    plt.legend(['train','val'],loc=4)
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])
    plt.show()

def load_data():
    # load your data using this function
    f = open("../dataset/myfood10-227.pkl", 'rb')

    d = pickle.load(f)
    data = d['trainFeatures']
    labels = d['trainLabels']
    lz = d['labels']
    data = data.reshape(data.shape[0], 3, 227, 227)
    #data = data.transpose(0, 2, 3, 1)

    return data,labels,lz
def load_model(nb_class, weights_path=None):

    inputs = Input(shape=(3,227,227))

    conv_1 = Convolution2D(96, 11, 11,subsample=(4,4),activation='relu',
                           name='conv_1')(inputs)

    conv_2 = MaxPooling2D((3, 3), strides=(2,2))(conv_1)
    conv_2 = crosschannelnormalization(name="convpool_1")(conv_2)
    conv_2 = ZeroPadding2D((2,2))(conv_2)
    conv_2 = merge([
        Convolution2D(128,5,5,activation="relu",name='conv_2_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_2)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_2")

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1,1))(conv_3)
    conv_3 = Convolution2D(384,3,3,activation='relu',name='conv_3')(conv_3)

    conv_4 = ZeroPadding2D((1,1))(conv_3)
    conv_4 = merge([
        Convolution2D(192,3,3,activation="relu",name='conv_4_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_4)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_4")

    conv_5 = ZeroPadding2D((1,1))(conv_4)
    conv_5 = merge([
        Convolution2D(128,3,3,activation="relu",name='conv_5_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_5)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_5")

    conv_5 = MaxPooling2D((3, 3), strides=(2,2),name="convpool_5")(conv_5)



    dense_1 = Flatten(name="flatten")(conv_5)
    dense_1 = Dense(4096, activation='relu',name='dense_1')(dense_1)
    dense_2 = Dropout(0.5)(dense_1)
    dense_2 = Dense(4096, activation='relu',name='dense_2')(dense_2)
    dense_3 = Dropout(0.5)(dense_2)
    dense_3 = Dense(nb_class,name='dense_3')(dense_3)
    prediction = Activation("softmax",name="softmax")(dense_3)


    base_model = Model(input=inputs, output=prediction)

    if weights_path:
        base_model.load_weights(weights_path)

    base_model = Model(input=inputs, output=dense_2)

    return base_model

def create_model(weights_path=None, heatmap=False):

    inputs = Input(shape=(3,227,227))

    conv_1 = Convolution2D(96, 11, 11,subsample=(4,4),activation='relu',
                           name='conv_1')(inputs)

    conv_2 = MaxPooling2D((3, 3), strides=(2,2))(conv_1)
    conv_2 = crosschannelnormalization(name="convpool_1")(conv_2)
    conv_2 = ZeroPadding2D((2,2))(conv_2)
    conv_2 = merge([
        Convolution2D(128,5,5,activation="relu",name='conv_2_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_2)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_2")

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1,1))(conv_3)
    conv_3 = Convolution2D(384,3,3,activation='relu',name='conv_3')(conv_3)

    conv_4 = ZeroPadding2D((1,1))(conv_3)
    conv_4 = merge([
        Convolution2D(192,3,3,activation="relu",name='conv_4_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_4)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_4")

    conv_5 = ZeroPadding2D((1,1))(conv_4)
    conv_5 = merge([
        Convolution2D(128,3,3,activation="relu",name='conv_5_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_5)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_5")

    dense_1 = MaxPooling2D((3, 3), strides=(2,2),name="convpool_5")(conv_5)


    dense_1 = Flatten(name="flatten")(dense_1)
    dense_1 = Dense(4096, activation='relu',name='dense_1')(dense_1)
    dense_2 = Dropout(0.5)(dense_1)
    dense_2 = Dense(4096, activation='relu',name='dense_2')(dense_2)
    dense_3 = Dropout(0.5)(dense_2)
    dense_3 = Dense(nb_classes,name='dense_3')(dense_3)
    prediction = Activation("softmax",name="softmax")(dense_3)


    model = Model(input=inputs, output=prediction)

    return model

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    opt = SGD(lr=0.01)
    model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
    #rms = RMSprop(lr=5e-4, rho=0.9, epsilon=1e-08, decay=0.01)
    #model.compile(optimizer=rms, loss='categorical_crossentropy', metrics=['accuracy'])
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    print "Training CNN.."
    hist = model.fit(X_train, Y_train, nb_epoch=nb_epoch, validation_data=(X_test, Y_test),batch_size=batch_size,verbose=1)
    visualize_loss(hist)
    model.save_weights("food10_scratch_weights.h5")
    
    scores = model.evaluate(X_test, Y_test, verbose=0)
    print("Softmax %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    
    model = None
    model = load_model(nb_class=nb_classes, weights_path="food10_scratch_weights.h5")
    
    svm_train = model.predict(X_train, batch_size=32)
    svm_test = model.predict(X_test, batch_size=32)
    
    print "\nTraining SVM.."
    clf = svm.SVC(kernel='rbf', gamma=0.7, C=1.0)

    clf.fit(svm_train, y_train.ravel())
    #y_pred = clf.predict(test_data)
    score = clf.score(svm_test, y_test.ravel())
    print("SVM %s: %.2f%%" % ("acc: ", score*100))
    
    y_pred = clf.predict(svm_test)
    target_names = ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
                     'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi']
    cm = confusion_matrix(y_test.ravel(), y_pred)
    print(cm)
    
    # Visualization of confusion matrix
    np.set_printoptions(precision=2)
    plt.figure()
    plot_confusion_matrix(cm)
    plt.show()
   
    
    # Compute confusion matrix
    #cm = confusion_matrix(np.argmax(y_test,axis=1), y_pred)
    #print cm
    #np.set_printoptions(precision=2)
    #plt.figure()
    #plot_confusion_matrix(cm)
    #plt.show()

    return score

 
if __name__ == "__main__":
    n_folds = 2
    cvscores = []
    data_augmentation = True
    print "Loading data.."
    data, labels, lz = load_data()
    print "Data loaded !"
    data = data.astype('float32')
    data /= 255
    lz = np.array(lz)
    total_scores = 0 


    skf = StratifiedKFold(y=lz, n_folds=n_folds, shuffle=True)

    for i, (train, test) in enumerate(skf):
        print ("Running Fold %d / %d" % (i+1, n_folds))
        model = None # Clearing the NN.
        model = create_model(weights_path=None, heatmap=False)
        scores = train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])
        #print "Score for fold ", i+1, "= ", scores*100
        total_scores = total_scores + scores

    print("Average acc : %.2f%%" % (total_scores/n_folds*100))


Using Theano backend.
Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5105)

Loading data..
Data loaded !
Running Fold 1 / 2


/home/machine/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')



Training CNN..
Train on 1000 samples, validate on 1000 samples
Epoch 1/200
1000/1000 [==============================] - 5s - loss: 2.3028 - acc: 0.1070 - val_loss: 2.3019 - val_acc: 0.1000
Epoch 2/200
1000/1000 [==============================] - 5s - loss: 2.3032 - acc: 0.0830 - val_loss: 2.3016 - val_acc: 0.1040
Epoch 3/200
1000/1000 [==============================] - 5s - loss: 2.3015 - acc: 0.1220 - val_loss: 2.3011 - val_acc: 0.1130
Epoch 4/200
1000/1000 [==============================] - 5s - loss: 2.3018 - acc: 0.1010 - val_loss: 2.3006 - val_acc: 0.1620
Epoch 5/200
1000/1000 [==============================] - 5s - loss: 2.3005 - acc: 0.1130 - val_loss: 2.3001 - val_acc: 0.1480
Epoch 6/200
1000/1000 [==============================] - 5s - loss: 2.2986 - acc: 0.1250 - val_loss: 2.2996 - val_acc: 0.1500
Epoch 7/200
1000/1000 [==============================] - 5s - loss: 2.3012 - acc: 0.1090 - val_loss: 2.2991 - val_acc: 0.2170
Epoch 8/200
1000/1000 [==============================]

/home/machine/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/machine/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')
